In [2]:
import numpy as np
from typing import Dict, Tuple, List
from copy import copy
from collections import defaultdict
from heapq import *
from min_cost_flow import Network

In [4]:
# Adapted from https://gist.github.com/kachayev/5990802
from collections import deque

def rev(edge):
    return (edge[1], edge[0], -edge[2])

def dijkstra(
    S_f: object,
    T_f: object,
    adj: Dict[Tuple[object, object], int]
) -> Tuple[Dict[object, int], List[Tuple[object, object]]]:
    '''
    Computes the shortest path distances from [s] to all other nodes 
    and the shortest path from [s] to [t] on the graph with edges/weights 
    given by the entries of [adj]. Implementation of Dijkstra's shortest
    path algorithm for graphs with non-negative edge costs.
    
    Args:
        s: Start node
        t: End node (for path)
        adj: Dictionary containing edges and their respective weights, with
            adj[(i, j)] = w_{ij} for nodes i and j.
    '''
    def _format_path(lst):
        return [(lst[i][0], *lst[i+1]) for i in range(len(lst) - 1)]
            
            
    # Generate underlying adjacency lists
    adjacency = defaultdict(list)
    for (i, j, id), c in adj.items():
        if c < 0:
            print(f"cost can't be < 0: {(i,j,id,c)}")
        assert(c >= 0)
        adjacency[i].append((j, id, c))

    # Initialize queue, distances
    queue, seen, distances = deque([(0, s, 0, []) for s in S_f]), set(), {s: 0 for s in S_f}
    
    while queue:
        cost, v1, id, path = heappop(queue)
        if v1 not in seen:
            seen.add(v1)
            path = [*path, (v1, id)]
            if v1 in T_f: 
                out_path = _format_path(path)

            for v2, id, c in adjacency.get(v1, ()):
                if v2 in seen: 
                    continue
                prev = distances.get(v2, None)
                nxt = cost + c
                if prev is None or nxt < prev:
                    distances[v2] = nxt
                    heappush(queue, (nxt, v2, id, path))
    return distances, out_path

In [5]:


def BFS(
        S_f: object,
        T_f: object,
        adj: List[Tuple[object, object]]
) -> Tuple[Dict[object, int], List[Tuple[object, object]]]:


    def _format_path(lst):
        return [(lst[i][0], *lst[i+1]) for i in range(len(lst) - 1)]

    # Generate underlying adjacency lists
    adjacency = defaultdict(list)
    for (i, j, id) in adj:
        adjacency[i].append((j, id))

    # Initialize queue, distances
    queue, seen = (deque([(s, 0, []) for s in S_f]), set())

    while queue:
        v1, id, path = queue[0]

        if v1 not in seen:
            seen.add(v1)
            path = [*path, (v1, id)]
            if v1 in T_f:
                out_path = _format_path(path)
                return out_path

            for v2, id in adjacency.get(v1, ()):
                if v2 in seen:
                    continue
                queue.append((v2, id, path))

        queue.popleft()
    return None

In [33]:
def reduced_cost(
    N: Network,
    u_f: Dict[Tuple[object, object], int],
    p: Dict[object, int]
) -> Dict[Tuple[object, object], int]:
    """
    Computes reduced costs of the edges in the residual graph [u_f] with respect to edge
    costs [N.c] and node potentials [p].

    Args:
        N: Network representing the problem input
        u_f: Dictionary encoding the residual graph w.r.t the current flow
        p: Current node potentials

    Returns:
        A dictionary which gives the reduced cost for each edge (u, v) according to
        c_p[(u, v)] = c[(u, v)] + p[u] - p[v].
    """
    reduced_costs = {}
    for e in u_f.keys():
        (u, v, _) = e
        if e in N.c:
            reduced_costs[e] = int(N.c[e] + p[u] - p[v])
        else:
            reduced_costs[e] = int(-N.c[rev(e)] + p[u] - p[v])
    return reduced_costs
    
def excess_nodes(
    N: Network,
    f: Dict[Tuple[object, object], int],
) -> Tuple[List, List, Dict]:
    '''
    Compute nodes in the network [N] where flow conservation is violated by at least [K] units
    for the flow [f]. 
    
    Args:
        N: Network object representing the problem input
        f: Potentially infeasible flow
    
    Returns:
        A tuple consisting of a list of nodes where net flow in is greater than [K]
        and a list of nodes where the net flow in is less than -[K].
    '''
    def _excess(N, f) -> Dict[object, int]:
        # Initialize excess to be supply
        excess = {v: N.b[v] for v in N.V}
        for (u, v, _), val in f.items():
            excess[u] -= val
            excess[v] += val

        return excess
    
    e_f = _excess(N, f)
    S_f = [i for (i, val) in e_f.items() if val > 0]
    T_f = [i for (i, val) in e_f.items() if val < 0]
    return S_f, T_f, e_f

def update_potentials(
    p: Dict[object, int],
    distances: Dict[object, int],
    P
) -> None:
    '''
    Updates node potentials [p] with the shortest path distances in
    [distances] according to p[i] <- p[i] + distances[i].
    
    Args:
        p: Previous node potentials
        distances: Shortest path distance to each node in graph from a node with
            surplus above the current scaling threshold
        
    '''
    t = P[-1][1]

    for i in p.keys():
        if i in distances:
            p[i] += (min(distances[i], distances[t]))
        else:
            p[i] += (distances[t])
                   
def saturate_edges(
    N: Network,
    f: Dict[Tuple[object, object], int],
    u_f: Dict[Tuple[object, object], int],
    edges: List[Tuple[object, object]]
) -> None:
    '''
    Updates the flow [f] and residual graph [u_f] by saturating
    all edges in [edges].
    
    Args:
        N: Flow network encoding the problem input
        u_f: The residual graph for the current flow
        f: The current flow
        edges: List of edges to saturate
        
    '''

    for e in edges:
        if e in f:
            f[e] = N.u[e]                              # Saturate foward edge
            u_f[e] = 0                                 # Zero forward residual edge
            u_f[rev(e)] = N.u[e]                       # Saturate backward residual edge

        else:
            f[rev(e)] = 0                              # Zero forward edge
            u_f[e] = 0                                 # Saturate forward residual edge
            u_f[rev(e)] = N.u[rev(e)]                  # Zero backward residual edge

        
def saturate_neg_cost_admissible(
    N: Network,
    c_p: Dict[Tuple[object, object], int],
    f: Dict[Tuple[object, object], int],
    u_f: Dict[Tuple[object, object], int]
) -> None:
    '''
    Updates the current flow [f] and residual graph [u_f] by
    saturating all edges with residual capacity of at least [K]
    and negative reduced cost [c_p] to preserve invariants in the
    algorithm.
    
    Args:
        N: Flow network encoding the problem input
        c_p: Current reduced costs
        f: The current flow
        u_f: The residual graph for the current flow
    '''
    neg_cost_admissible = [
        e
        for e, u in u_f.items() 
        if c_p[e] < 0 and u > 0
    ]
    print(f"Number of negative cost admissible edges: {len(neg_cost_admissible)}")
    
    saturate_edges(N, f, u_f, neg_cost_admissible)
    
def augment_flow_along_path(
    P: List[Tuple[object, object]],
    f: Dict[Tuple[object, object], int],
    u_f: Dict[Tuple[object, object], int],
    e_f
) -> None:
    ''' 
    Updates the current flow [f] and residual graph [u_f] by
    pushing [K] units of flow along the directed path P.
    
    Args:
        P: Path of edges to push flow
        f: Current flow
        c_f: Current residual graph
        K: Scaling parameter
    
    '''
    s = P[0][0]
    t = P[-1][1]

    min_capacity = min(u_f[e] for e in P)
    delta = min(e_f[s], -e_f[t], min_capacity)
    
    for e in P:
        if e in f:
            f[e] += delta
            u_f[e] -= delta
            u_f[rev(e)] = u_f.get(rev(e), 0) + delta
            
        else:
            f[rev(e)] -= delta
            u_f[rev(e)] += delta
            u_f[e] -= delta

def primal_value(N, f):
    return np.sum([f[e]*N.c[e] for e in N.E if e in f])

def dual_value(N, p):
    return -np.sum([p[i] * N.b[i] for i in N.V]) - np.sum([N.u[e] * max(0, p[e[1]] - p[e[0]] - N.c[e]) for e in N.E])

def init_residual_graph(N, f):
    u_f = {}
    for e in N.E:
        if e in f:
            u_f[e] = N.u[e] - f[e]
            u_f[rev(e)] = f[e]
        else:
            u_f[e] = N.u[e]
    return u_f        
    
def compute_feasible_flow(
    N: Network,
    p
):
    f = {e: 0 for e in N.E}
    u_f = init_residual_graph(N, f)
    c_p = reduced_cost(N, u_f, p)
    saturate_neg_cost_admissible(N, c_p, f, u_f)
    N_prime = copy(N)
    N_prime.E = [e for e in N.E if c_p[e] == 0]

    u_f_prime = init_residual_graph(N_prime, f)
          
    S_f, T_f, e_f = excess_nodes(N_prime, f)

    while len(S_f) > 0:

        # Admissible edges
        adj = [e for (e, u) in u_f_prime.items() if u > 0]
        P = BFS(S_f, T_f, adj)
        if P is None:
            break
        augment_flow_along_path(P, f, u_f_prime, e_f)
        S_f, T_f, e_f = excess_nodes(N_prime, f)

    return f
    
def successive_shortest_paths(
    N: Network,
    **kwargs
) -> Tuple[Dict[Tuple[object, object], int], Dict[object, int]]:
    '''
    Primal-dual algorithm for computing a minimum-cost flow for the 
    network [N] starting from dual-feasible node potentials [p].
    
    Args:
        N: Flow network encoding the problem input
        p: Initial node potentials for warm start
        
    Returns:
        Minimum cost flow and corresponding optimal node potentials
    '''
    
    # Init zero flow and potentials
    if 'f' not in kwargs:
        f = {e: 0 for e in N.E}
    else:
        f = copy(kwargs['f'])

    if 'p' not in kwargs:
        p = {v: 0 for v in N.V}
        f = {e: 0 for e in N.E}
    else:
        p = copy(kwargs['p'])
        f = compute_feasible_flow(N, p)

    iter_limit = kwargs.get('iter_limit', -1)

    u_f = init_residual_graph(N, f)

    iters=0
    
    # Compute reduced costs w.r.t potentials p
    c_p = reduced_cost(N, u_f, p)
    S_f, T_f, e_f = excess_nodes(N, f)


    while len(S_f) > 0:
        # print(f"iteration: {iters}, excess: {sum(np.abs(list(e_f.values())))}, primal value: {primal_value(N, f)}, dual value: {dual_value(N, p)}")

        # Admissible edges
        adj = {e: c for (e, c) in c_p.items() if u_f[e] > 0}
        D, P = dijkstra(S_f, T_f, adj)
        update_potentials(p, D, P)
        augment_flow_along_path(P, f, u_f, e_f)
        c_p = reduced_cost(N, u_f, p)
        S_f, T_f, e_f = excess_nodes(N, f)

        iters += 1
        if iters % 10 == 0:
            print(f"Current iteration number: {iters}, flow value: {primal_value(N, f)}")
        if iters == iter_limit:
            return False, f, p

    assert len({e: c for (e, c) in c_p.items() if u_f[e] > 0 and c < 0}) == 0
        
    #print(f"end reduced costs:\n{c_p}")
    #print(f"end flow:\n{f}")
    # for e in N.E:
        # print(f"{e}: cost: {c_p[e]}, flow: {f[e]}")
    print(f"number of edges: {len(c_p)}")
    print(f"Number of flow updates: {iters}, final flow value: {primal_value(N, f)}")
    return True, f, p

In [7]:
def feasibility_check(N, f):
    assert np.all(np.array([len(excess_nodes(N, f)[i]) == 0 for i in [0,1]]))
    assert np.all(np.array(list(f.values())) >= 0)
    assert np.all(np.array(list(f.values())) <= np.array(list(N.u.values())))
    
def optimality_check(N, f, p):
    primal = np.sum([f[e]*N.c[e] for e, _ in f.items()])
    dual = -np.sum([p[i] * N.b[i] for i in N.V]) - np.sum([N.u[e] * max(0, p[e[1]] - p[e[0]] - N.c[e]) for e in N.E])
    print(primal)
    print(dual)
    assert np.isclose(primal, dual, atol=0.0)
    return primal


In [8]:
edges = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,2)]
capacities = np.array([15, 8, 20, 4, 10, 15, 4, 20, 5])
costs = np.array([4, 4, 2, 2, 6, 1, 3, 2, 3])
supplies = [20, 0, 0, -5, -15]
nodes = [0, 1, 2, 3, 4]
N = Network(nodes, edges, capacities, costs, supplies)     

f, p = successive_shortest_paths(N)

number of edges: 24
Number of flow updates: 5, final flow value: 150


In [9]:
f, p = successive_shortest_paths(N, p=p)

Number of negative cost admissible edges: 3
number of edges: 24
Number of flow updates: 0, final flow value: 150.0


In [10]:
feasibility_check(N, f)
optimality_check(N, f, p)

150.0
150.0


150.0

In [11]:
def parse(filename) -> Network:
    """
    Parses a network file following the DIMACS problem specification 
    structure and transforms it into a Network object
    
    Some elements of the specification:
    - Lines starting in c are comments
    - Lines starting in p explain what problem to solve (can be ignored, 
      we only consider minimum-cost flow problems)
    - Lines starting in n define nodes
    - Lines starting in a define arcs (edges)
    
    Args:
        filename: name of the file containing the network data
        
    Returns:
        The corresponding Network object
    """
    # Lines we can ignore
    ignore_list = ['c', 'p']
    
    file = open(filename, 'r')
    
    # Nodes is a hashmap from node values to their supply
    nodes = {}
    # Edges is a hashmap from edges to a tuple with their capacity and cost
    edges = {}
    
    for line in file:
        if len(line) > 0 and line[0] not in ignore_list:
            if line[0] == 'n':
                # Node parsing
                node = [int(elem) for elem in line.split(' ')[1:]]
                nodes[node[0]] = node[1]
            elif line[0] == 'a':
                arc = [int(elem) for elem in line.split(' ')[1:]]
                node1 = arc[0]
                node2 = arc[1]
                capacity = arc[3]
                cost = arc[4]
                
                # Only nodes with non-zero supply are in a "node line"
                if node1 not in nodes:
                    nodes[node1] = 0
                if node2 not in nodes:
                    nodes[node2] = 0
                if (node1, node2) in edges:
                    # TODO not amazing (reaverages every time)
                    old_capacity, old_cost = edges[(node1, node2)]
                    new_cost = old_cost * old_capacity + cost * capacity
                    new_cost /= (old_capacity + capacity)
                    edges[(node1, node2)] = (old_capacity + capacity, new_cost)
                else:
                    edges[(node1, node2)] = (capacity, cost)
    file.close()
    
    capacities, costs = zip(*edges.values())
    network = Network(list(nodes.keys()), list(edges.keys()), capacities, costs, list(nodes.values())) 
    
    print(f"This dataset contains: {len(nodes.keys())} nodes and {len(edges.keys())} edges")

    return network

In [12]:
%%time
network = parse("resources/road_flow_01_DC_a.txt")
# f, p = successive_shortest_paths(network, iter_limit=5)
f, p = successive_shortest_paths(network)

This dataset contains: 9559 nodes and 29682 edges
number of edges: 59396
Number of flow updates: 62, final flow value: 96478500.0
CPU times: user 7.1 s, sys: 30.9 ms, total: 7.13 s
Wall time: 7.15 s


In [13]:
feasibility_check(network, f)
optimality_check(network, f, p)

96478500.0
96478440.0


96478500.0

In [14]:
%%time
f2, p2 = successive_shortest_paths(network, p=p)

Number of negative cost admissible edges: 200
number of edges: 59396
Number of flow updates: 0, final flow value: 96478500.0
CPU times: user 3.17 s, sys: 0 ns, total: 3.17 s
Wall time: 3.17 s


In [15]:
feasibility_check(network, f2)
optimality_check(network, f2, p2)

96478500.0
96478440.0


96478500.0

In [16]:
f2

{(9558, 7231, 1): 0,
 (7231, 9558, 2): 0,
 (7235, 9558, 3): 120.0,
 (9558, 7235, 4): 0,
 (7231, 7080, 5): 0,
 (7080, 7231, 6): 0,
 (7235, 7231, 7): 0,
 (7231, 7235, 8): 0,
 (4908, 9559, 9): 0,
 (9559, 4908, 10): 0,
 (9559, 4909, 11): 0,
 (4909, 9559, 12): 0,
 (9559, 7178, 13): 0,
 (7178, 9559, 14): 0,
 (9347, 9342, 15): 0,
 (9342, 9347, 16): 0,
 (9558, 7232, 17): 120.0,
 (7232, 9558, 18): 0,
 (7232, 7174, 19): 120.0,
 (7174, 7232, 20): 0,
 (7176, 7232, 21): 0,
 (7232, 7176, 22): 0,
 (5891, 5890, 23): 120.0,
 (5890, 5891, 24): 0,
 (6019, 6515, 25): 0,
 (6515, 6019, 26): 0,
 (9376, 6020, 27): 0,
 (6020, 9376, 28): 0,
 (6021, 6023, 29): 0,
 (6023, 6021, 30): 0,
 (5740, 5782, 31): 0,
 (5782, 5740, 32): 480.0,
 (9557, 5738, 33): 0,
 (5738, 9557, 34): 0,
 (5724, 9557, 35): 0,
 (9557, 5724, 36): 0,
 (5723, 5730, 37): 0,
 (5730, 5723, 38): 0,
 (9553, 1268, 39): 0,
 (1268, 9553, 40): 0,
 (9556, 5781, 41): 0,
 (5781, 9556, 42): 0,
 (9555, 5570, 43): 0,
 (5570, 9555, 44): 0,
 (9554, 9553, 45): 0,

In [30]:
%%time
network = parse("data/netgen_lo_sr_11d.min")
# f, p = successive_shortest_paths(network, iter_limit=5)
f, p = successive_shortest_paths(network)


This dataset contains: 2048 nodes and 92682 edges
Current iteration number: 10, flow value: 9655
Current iteration number: 20, flow value: 53547
Current iteration number: 30, flow value: 104471
Current iteration number: 40, flow value: 139405
Current iteration number: 50, flow value: 182276
Current iteration number: 60, flow value: 216289
Current iteration number: 70, flow value: 259916
Current iteration number: 80, flow value: 307227
Current iteration number: 90, flow value: 376275
Current iteration number: 100, flow value: 446320
Current iteration number: 110, flow value: 524987
Current iteration number: 120, flow value: 587385
Current iteration number: 130, flow value: 684536
number of edges: 185544
Number of flow updates: 134, final flow value: 715310
CPU times: user 33.8 s, sys: 4 ms, total: 33.8 s
Wall time: 33.8 s
